In [1]:
# Importing necessary libraries
import pandas as pd
from collections import defaultdict, deque
import heapq

df = pd.read_csv('tubedata.csv', header=None)
df.head()

##print(df.head())
 
station_dict = defaultdict(list)
zone_dict = defaultdict(set)

# get data row by row
for index, row in df.iterrows():
  
  start_station = row[0]
  end_station = row[1]
  act_cost = int(row[3])

  zone1 = row[4]
  zone2 = row[5]

  # station dictionary of child station tuples (child_name, cost from parent to the child)
  # {"Mile End": [("Stepney Green", 2), ("Wembley", 1)]}
  station_list = station_dict[start_station]
  station_list.append((end_station, act_cost))

  # the following two lines add the other direction of the tube "step"
  station_list = station_dict[end_station]
  station_list.append((start_station, act_cost))
  
  # we add the main zone
  zone_dict[start_station].add(zone1)
  # we add the secondary zone
  if zone2 != "0":
    zone_dict[start_station].add(zone2)
    # if the secondary zone is not 0 it's the main zone for the ending station
    zone_dict[end_station].add(zone2)
  else:
    # otherwise the main zone for the ending station is the same as for the starting station
    zone_dict[end_station].add(zone1)
# Setting the initial conditions or parameters

## Question 2.1 - Implement DFS, BFS and UCS

In [2]:
# Loading and preprocessing data
def dfs(graph, start, goal, inverse_order=False):
    visited_nodes = set()
    store_list = [(start, [start], 0)]  # station, path, cost
    exp_nodes = 0

    while store_list:
        (station, path, cost) = store_list.pop()
        exp_nodes += 1

        if station == goal:
            return path, cost, exp_nodes
        visited_nodes.add(station)

        neighbors = graph[station]
        if inverse_order:
            neighbors = reversed(neighbors)

        for next, next_cost in neighbors:
            if next not in visited_nodes:
                store_list.append((next, path + [next], cost + next_cost))

    return None


def bfs(graph, start, goal, inverse_order=False):
    queue = deque([(start, [start])]) 
    visited = set()

    while queue:
        current, path = queue.popleft()

        if current == goal:
            return path  

        if current not in visited:
            visited.add(current)
            neighbors = graph.get(current, [])

            if inverse_order:
                neighbors = reversed(neighbors)

            for neighbor in neighbors:
                neighbor_name, _ = neighbor
                if neighbor_name not in visited:
                    queue.append((neighbor_name, path + [neighbor_name]))

    return None

def ucs(graph, start_station, goal_station, zone_dict, inverse_order=False):
    start_state = (0, start_station, [start_station])  # cost, station, path
    priority_queue = [start_state]
    visited = set()

    while priority_queue:
        current_cost, current_station, current_path = heapq.heappop(priority_queue)

        if current_station == goal_station:
            return current_path, current_cost, len(visited) + 1  # +1 for the current node

        if current_station not in visited:
            visited.add(current_station)

            neighbors = graph[current_station]
            if inverse_order:
                neighbors = sorted(neighbors, key=lambda x: x[1], reverse=True)

            for neighbor, cost in neighbors:
                if neighbor not in current_path:
                    new_cost = current_cost + cost
                    new_state = (new_cost, neighbor, current_path + [neighbor])
                    heapq.heappush(priority_queue, new_state)

    return None

# Cleaning and organising data for analysis

## Question 2.3 - Extending the cost function

In [3]:
# General code processing
lines_and_stations_dict = defaultdict(list)

for index, row in df.iterrows():
    start_station, end_station, line, act_cost, _, _ = row
    act_cost = int(act_cost)

    # Adding line information to the station dictionary
    lines_and_stations_dict[start_station].append((end_station, (act_cost, line)))
    lines_and_stations_dict[end_station].append((start_station, (act_cost, line)))


def ucs_with_line_change(graph, start, goal, line_change_cost=2):
    queue = [(0, start, [start], None)]  # Cost, station, path, current line
    expanded_nodes = 0
    visited_nodes = set()

    while queue:
        cost, edge, path, current_line = heapq.heappop(queue)
        expanded_nodes += 1

        if edge == goal:
            return path, cost, expanded_nodes
        if edge not in visited_nodes:
            visited_nodes.add(edge)
            
        for next_station, next_info in graph[edge]:
            next_cost, new_line = next_info # Assuming the graph stores cost and line
            total_cost = cost + next_cost

            # Check for line change and add penalty if needed
            if current_line and new_line != current_line:
                total_cost += line_change_cost

            if next_station not in path:
                heapq.heappush(queue, (total_cost, next_station, path + [next_station], new_line))

    return None

def ucs_with_line_change_final(graph, start_station, end_station):
    # UCS with time change implementation
    ucs_path, ucs_start, ucs_end = ucs_with_line_change(lines_and_stations_dict, start_station, end_station)
    print("UCS with line changes:", ucs_path, ucs_start, ucs_end)
    
def search_algorithms_combined(graph, start_station, end_station, inverse):
    print("Searching routes from ", start_station, "to", end_station)

    # DFS - Depth-First Search
    dfs_path, dfs_start, dfs_end = dfs(graph, start_station, end_station, inverse)
    print("DFS:", dfs_path, dfs_start, dfs_end)

    # BFS - Breadth-First Search
    bfs_path = bfs(graph, start_station, end_station, inverse)
    bfs_tot_cost = sum(graph[node1][0][1] for node1 in bfs_path[1:])
    bfs_path_expanded = len(bfs_path)
    print("BFS:", bfs_path, bfs_tot_cost, bfs_path_expanded)
    
    # UCS - Uniform Cost Search
    ucs_path, ucs_start, ucs_end = ucs(graph, start_station, end_station, zone_dict, inverse)
    print("UCS :", ucs_path, ucs_start, ucs_end)



In [4]:
# General code processing
search_algorithms_combined(station_dict, 'New Cross Gate', "Stepney Green", True) ## If Inverse is True

Searching routes from  New Cross Gate to Stepney Green
DFS: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Aldgate East', 'Tower Hill', 'Aldgate', 'Liverpool Street', 'Bethnal Green', 'Mile End', 'Stepney Green'] 28 358
BFS: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green'] 12 8
UCS : ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green'] 14 19


In [5]:
search_algorithms_combined(station_dict, 'New Cross Gate', "Stepney Green", False) ## When Inverse if false

Searching routes from  New Cross Gate to Stepney Green
DFS: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford', 'Mile End', 'Stepney Green'] 27 35
BFS: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green'] 12 8
UCS : ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green'] 14 19


In [6]:
ucs_with_line_change_final(station_dict, 'New Cross Gate', "Stepney Green" ) ## UCS with the line Change

UCS with line changes: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green'] 16 17


In [7]:

search_algorithms_combined(station_dict, 'Euston', "Victoria", False)

Searching routes from  Euston to Victoria
DFS: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'] 7 5
BFS: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'] 7 5
UCS : ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria'] 7 35


In [8]:

search_algorithms_combined(station_dict, 'Canada Water', "Stratford", False)

Searching routes from  Canada Water to Stratford
DFS: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford'] 15 6
BFS: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford'] 15 6
UCS : ['Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green', 'Mile End', 'Stratford'] 14 55


In [9]:

search_algorithms_combined(station_dict, 'Baker Street', "Wembley Park", False)

Searching routes from  Baker Street to Wembley Park
DFS: ['Baker Street', 'Finchley Road', 'Wembley Park'] 13 3
BFS: ['Baker Street', 'Finchley Road', 'Wembley Park'] 5 3
UCS : ['Baker Street', 'Finchley Road', 'Wembley Park'] 13 85


In [10]:

search_algorithms_combined(station_dict, 'Hammersmith', "Upminster", False)

Searching routes from  Hammersmith to Upminster
DFS: ['Hammersmith', 'Turnham Green', 'Acton Town', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Great Portland Street', 'Euston Square', "King's Cross St. Pancras", 'Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria', 'Pimlico', 'Vauxhall', 'Stockwell', 'Oval', 'Kennington', 'Elephant & Castle', 'Borough', 'London Bridge', 'Bank/Monument', 'Moorgate', 'Liverpool Street', 'Aldgate', 'Tower Hill', 'Aldgate East', 'Whitechapel', 'Stepney Green', 'Mile End', 'Bow Road', 'Bromley-by-Bow', 'West Ham', 'Plaistow', 'Upton Park', 'East Ham', 'Barking', 'Upney', 'Becontree', 'Dagenham Heathway', 'Dagenham East', 'Elm Park', 'Hornchurch', 'Upminster Bridge', 'Upminster'] 142 162
BFS: ['Hammersmith', 'Barons Court', "Earls' Court", 'Glou

In [11]:

search_algorithms_combined(station_dict, 'Ealing Broadway', "South Kensington", False)

Searching routes from  Ealing Broadway to South Kensington
DFS: ['Ealing Broadway', 'Ealing Common', 'North Ealing', 'Park Royal', 'Alperton', 'Sudbury Town', 'Sudbury Hill', 'South Harrow', 'Rayners Lane', 'West Harrow', 'Harrow-on-the-Hill', 'Northwick Park', 'Preston Road', 'Wembley Park', 'Finchley Road', 'Baker Street', 'Great Portland Street', 'Euston Square', "King's Cross St. Pancras", 'Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria', 'Sloane Square', 'South Kensington'] 67 194
BFS: ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'] 16 9
UCS : ['Ealing Broadway', 'Ealing Common', 'Acton Town', 'Turnham Green', 'Hammersmith', 'Barons Court', "Earls' Court", 'Gloucester Road', 'South Kensington'] 19 50


## Question 2.4 - Heuristic search

In [12]:
from queue import PriorityQueue

def heuristic(station, goal_station, zone_dict):
    # Check if both the current and goal stations are in the zone dictionary
    if station in zone_dict and goal_station in zone_dict:
        station_zones = zone_dict[station]
        goal_zones = zone_dict[goal_station]
        # Convert zone information to integers for calculation
        station_zones = {int(zone) for zone in station_zones}
        goal_zones = {int(zone) for zone in goal_zones}

        # Calculate the minimum zone difference between current and goal stations
        min_zone_difference = min(abs(station_zone - goal_zone) for station_zone in station_zones for goal_zone in goal_zones)

        return min_zone_difference
    else:
        return 0  

def best_first_search(graph, start, goal, zone_dict):
    priority_list = PriorityQueue()
    visited = set()
    # Initialise the priority queue with the start nod
    priority_list.put((heuristic(start, goal, zone_dict), start, [start]))

    while not priority_list.empty():
        _, edge, path = priority_list.get()
        # Return the path if the goal is reached
        if edge == goal:
            return path
        # Process the neighbors of the current node if it has not been visited
        if edge in graph and edge not in visited:
            visited.add(edge)
            for neighbor, _ in graph[edge]:
                if neighbor not in path:
                     # Add neighbors to the priority queue with their heuristic values
                    priority_list.put((heuristic(neighbor, goal, zone_dict), neighbor, path + [neighbor]))

    return None

start_station = "New Cross Gate"
goal_station = "Stepney Green"
bfs_path = best_first_search(station_dict, start_station, goal_station, zone_dict)
print("\nBest-First Search:")
print("Path:", bfs_path)


Best-First Search:
Path: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']
